In [1]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint
import keras.metrics
import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
sequence_length = 25 # timestep
# choose_joints = np.array([ 8, 6, 5, 12, 10, 9, 15, 14, 13, 19, 18, 17, 4, 3 ]) - 1
choose_joints = np.array([ 8, 6, 5, 12, 10, 9]) - 1
select_column = []
for i in range(len(choose_joints)): # 14 body join( except waist)
    select_column.append(0 + 3*choose_joints[i]) # select x
    select_column.append(1 + 3*choose_joints[i]) # select y
    select_column.append(2 + 3*choose_joints[i]) # select z 

In [3]:
path_save = "F:/Master Project/Dataset/Extract_Data/25 joints"
f_x = open(path_save+"/train_x.pickle",'rb')
f_y = open(path_save+"/train_y.pickle",'rb')
origin_train_x = pickle.load(f_x)
origin_train_y = np.array(pickle.load(f_y))

f_x = open(path_save+"/test_x.pickle",'rb')
f_y = open(path_save+"/test_y.pickle",'rb')
origin_test_x = pickle.load(f_x)
origin_test_y = np.array(pickle.load(f_y))

In [12]:
print(origin_train_x[0].shape)
print(len(origin_test_x))

(66330, 18)
6


### Reshape Input

In [4]:
def reform25to14(data):
    new_data = []
    # data shape (file,frame,75)
    for file in data:
        new_file = file[:,select_column]
#         import pdb;pdb.set_trace()
        new_data.append(new_file)
    return new_data

In [5]:
origin_train_x = reform25to14(origin_train_x)
origin_test_x = reform25to14(origin_test_x)

In [ ]:
# origin_test_x[0].shape

### Sampling Function

In [6]:
def sampling_x(x):
    
    frames = x
    
    random_sample_range = 3 # sampling value is not more than 3

    # Randomly choose sample interval and start frame
    sample_interval = np.random.randint(1, random_sample_range + 1)

    start_i = np.random.randint(0, len(frames) - sample_interval * sequence_length + 1)

    # Extract frames as tensors
    image_sequence = []
    end_i = sample_interval * sequence_length + start_i
    for i in range(start_i, end_i, sample_interval):
        # image_path = frames[i]
        if len(image_sequence) < sequence_length:
            image_sequence.append(frames[i])
        else:
            break
    image_sequence = np.array(image_sequence)   
    return image_sequence

# Use for sampling and reforming data for sending to ML model
def reform_to_sequence(data_x, data_y, is_training):
    
    if is_training:
        random_time = 20000
        output_x = np.zeros((len(data_x)*random_time, sequence_length, data_x[0].shape[-1]) ) #(len,timestep, 28)
        
    else:        
        random_time = 10000
        output_x = np.zeros((len(data_x)*random_time, sequence_length, data_x[0].shape[-1]) ) #(len*random_time,timestep, 28)
    
    count = 0
    output_y = np.arange( len(data_y)*random_time ) # create array
    
    # sampling window-data in random_time time
    for n_time in range(random_time):
        for i,x in enumerate(data_x):
            sequence = sampling_x(x)
            output_x[count] = sequence
            output_y[count] = data_y[i]
            count += 1
    
    # output_x - x_data   : shape(num_of_file * random_time, sequence_length, 75)
    # output_y - y_data   : shape(num_of_file * random_time, sequence_length, 75)
    return output_x, output_y     


### Model Define

In [7]:
def create_model(sequence_length, number_feature):
    model = Sequential()
    model.add(CuDNNLSTM(50, input_shape=(sequence_length, number_feature),return_sequences=False))
    model.add(Dropout(0.4))#使用Dropout函数可以使模型有更多的机会学习到多种独立的表征
    model.add(Dense(60) )
    model.add(Dropout(0.4))
    model.add(Dense(4, activation='softmax'))
    return model

In [9]:
number_feature = origin_train_x[0].shape[-1] # last index

load_model = False
model = create_model(sequence_length, number_feature)
start_epoch = 0

if load_model:
    weights_path = 'weight-sampling-01-0.95.hdf5'    
    model.load_weights(weights_path)

sgd = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
filepath="weight-sampling-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True,)
callbacks_list = [checkpoint]

### Prepare Test Set

In [10]:
test_x, test_y  = reform_to_sequence(origin_test_x, origin_test_y, is_training=False)

In [ ]:
len(test_x)

In [ ]:
# train_x, train_y = reform_to_sequence(origin_train_x, origin_train_y, is_training=True)

In [ ]:
# train_y.shape

### Train

In [13]:
num_epoch = 100

for i_ep in range(start_epoch+1,num_epoch):
    
    print('epoch: ', i_ep)
    train_x, train_y = reform_to_sequence(origin_train_x, origin_train_y, is_training=True)
    model.fit(train_x, train_y, epochs=start_epoch+1,
             validation_data=(test_x,test_y), callbacks=callbacks_list, initial_epoch=start_epoch)
    print("-----------------------")



epoch:  1
Train on 140000 samples, validate on 60000 samples
139808/140000 [============================>.] - ETA: 0s - loss: 0.7384 - accuracy: 0.7201
Epoch 00001: val_accuracy improved from -inf to 0.76870, saving model to weight-sampling-01-0.77.hdf5
140000/140000 [==============================] - 35s 248us/sample - loss: 0.7379 - accuracy: 0.7203 - val_loss: 0.5850 - val_accuracy: 0.7687
-----------------------
epoch:  2
Train on 140000 samples, validate on 60000 samples
139872/140000 [============================>.] - ETA: 0s - loss: 0.4556 - accuracy: 0.8400
Epoch 00001: val_accuracy improved from 0.76870 to 0.89413, saving model to weight-sampling-01-0.89.hdf5
140000/140000 [==============================] - 36s 259us/sample - loss: 0.4556 - accuracy: 0.8400 - val_loss: 0.3135 - val_accuracy: 0.8941
-----------------------
epoch:  3
Train on 140000 samples, validate on 60000 samples
139904/140000 [============================>.] - ETA: 0s - loss: 0.4202 - accuracy: 0.8539
Epoch

-----------------------
epoch:  22
Train on 140000 samples, validate on 60000 samples
139936/140000 [============================>.] - ETA: 0s - loss: 0.1001 - accuracy: 0.9685
Epoch 00001: val_accuracy did not improve from 0.94885
140000/140000 [==============================] - 47s 338us/sample - loss: 0.1001 - accuracy: 0.9685 - val_loss: 0.2666 - val_accuracy: 0.9123
-----------------------
epoch:  23
Train on 140000 samples, validate on 60000 samples
139968/140000 [============================>.] - ETA: 0s - loss: 0.0806 - accuracy: 0.9750
Epoch 00001: val_accuracy did not improve from 0.94885
140000/140000 [==============================] - 48s 341us/sample - loss: 0.0807 - accuracy: 0.9750 - val_loss: 0.3991 - val_accuracy: 0.9050
-----------------------
epoch:  24
Train on 140000 samples, validate on 60000 samples
139872/140000 [============================>.] - ETA: 0s - loss: 0.0745 - accuracy: 0.9767
Epoch 00001: val_accuracy did not improve from 0.94885
140000/140000 [=====

KeyError: 'val_accuracy'